Import dependencies

In [1]:
import data_prep as prep
import random
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from keras import layers
from tensorflow.keras.callbacks import EarlyStopping

Set variables

In [2]:
words = ['deaf', 'eat', 'fish', 'friend', 'like', 'milk', 'nice', 'no', 'orange', 'teacher', 'want', 'what', 'where', 'white', 'yes']
select_words = ['eat', 'teacher', 'want']
path = '../../preprocessing/dataset/'
detector_path = '../models/hand_landmarker.task'
num_features = 126
model_name = 'draft_model'
fps = 20

Get data from mediapipe


In [3]:
X, y, num_videos, highest_frame, bad_videos = prep.get_data(select_words, path, detector_path)

print('Number of videos:', num_videos)
print('Highest frame:', highest_frame)
print('Videos with no landmarkers detected: ', bad_videos)


  0%|          | 0/3 [00:00<?, ?it/s]

eat:   0%|          | 0/98 [00:00<?, ?it/s]

teacher:   0%|          | 0/101 [00:00<?, ?it/s]

want:   0%|          | 0/80 [00:00<?, ?it/s]

Number of videos: 251
Highest frame: 230
Videos with no landmarkers detected:  28


Padding and Masking X

In [4]:
padded_X, mask = prep.padX(X, num_videos, highest_frame, num_features)
print(padded_X.shape)

(251, 230, 126)


Split data

In [5]:
X_train, X_test, y_train, y_test = train_test_split(padded_X, y, test_size=0.2, random_state=42)


X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

Create model

In [6]:
model = keras.Sequential()

model.add(keras.Input(shape=(highest_frame, num_features)))
model.add(layers.Masking(mask_value=0.0))
model.add(layers.LSTM(64))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(select_words), activation='sigmoid'))


model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 230, 126)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        48,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,075 (199.51 KB)

 Trainable params: 51,075 (199.51 KB)

 Non-trainable params: 0 (0.00 B)

Train model

In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

model.fit(X_train, y_train, epochs=100, callbacks=[early_stopping])

model.save(f'../models/{model_name}.keras')

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 0.3224 - loss: 1.1215
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.4404 - loss: 1.0381
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.4890 - loss: 1.0003
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.5030 - loss: 0.9270
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.6236 - loss: 0.9025
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.6528 - loss: 0.8275
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6136 - loss: 0.8110
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.7155 - loss: 0.7300
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.6958 - loss: 0.7397
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.7126 - loss: 0.7290
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.7129 - loss: 0.6581
Epoch 12/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.6624 - lo

Evaluate model

In [8]:
results = model.evaluate(X_test, y_test)

print('Test loss:', results)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.8276 - loss: 0.4925 
Test loss: [0.4995381832122803, 0.8039215803146362]


Use model

In [9]:
i = random.randint(0,X_test.shape[0]-1)


X_prediction = X_test[i,:,:]
y_prediction = select_words[y_test[i]]

print(model.predict(np.array([X_prediction])))
print("should be", y_prediction)
print("predicted", select_words[np.argmax(model.predict(np.array([X_prediction])))])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
[[0.5881078  0.8552418  0.27551687]]
should be teacher
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
predicted teacher


Get test accuracy for each word

In [10]:
dic = prep.get_word_accuracy(select_words, model, X_test, y_test)

for key in dic:
    print(f"{key}: {dic[key][0]}/{dic[key][1]}")

eat: 17/22
teacher: 14/15
want: 10/14


Save model info

In [11]:
with open(f"../models/{model_name}.env", "w") as file:
    file.write(f"MAX_FRAMES={highest_frame}\n")
    file.write(f"NUM_FEATURES={num_features}\n")
    file.write(f"WORDS={",".join(select_words)}\n")
    file.write(f"FPS={fps}\n")
    file.write(f"TEST_ACC={results[1]}\n")
    file.write(f'WORD_ACC="{dic}"\n')